In [ ]:
program shoot  % 打靶法求例题
parameter (m=2,n=100,eps=0.00001)
real(8) h, z(m,n), y(m), d(m), x1, xn, yb, alfa, beta, g1, g2
integer i,j
!open(2, file='shoot1.txt')   % 有空气阻力
open(2, file='shoot0.txt')    % 没有空气阻力
alfa = 0.0; beta = 40.0; g1 = 30.0
x1 = 0.0; xn = 5.0  % a,b
h = (xn - x1) / (n - 1)  % 步长
y(1) = alfa
y(2) = g1
call rk4(x1, y, m, h, n, z, d)
yb = z(1,n)
y(1) = alfa
y(2)=beta*g1/z(1,n)
g2=y(2)
call rk4(x1,y,m,h,n,z,d)
if (abs(z(1,n)-beta)>eps) then
   y(1)=alfa
   y(2)=g2+(beta-z(1,n))*(g2-g1)/(z(1,n)-yb)
   g1=g2
   g2=y(2)
   yb=z(1,n)
   call rk4(x1,y,m,h,n,z,d)
end if
print *, 'z(2,1)=' , z(2,1)
do i=1,n
   x=x1+(i-1)*h
   write(2,'(2f14.8)') x, z(1,i)
end do
end
subroutine f(x, y, m, d)  ! y(1): 空间高度，y(2): 速度，x: 时间
real(8) y(m), d(m), x
d(1) = y(2)
d(2) = -9.8  ! 没有空气阻力
! d(2) = -9.8 - 0.01*y(2)  ! 有空气阻力
end


### 求解量子力学一维无限深势阱**

In [ ]:
function tt = shootev()
clc; clear; format long; global k
k = 1; tol = 1e-8; m = 3;
kk = zeros(1, m); % Pre-allocate kk

for n = 1:m      % Finding the first m eigenvalues
    dk = k / 20;   % Initial step size
    k = k + dk;     % First trial
    [x, phi] = ode45(@evfun, [0, 1], [0, 1e-3]);
    oldphi = phi(end, 1);
    dphi = oldphi;

    while abs(dphi) > tol
        k = k + dk; % Try a new k
        [x, phi] = ode45(@evfun, [0, 1], [0, 1e-3]);
        dphi = phi(end, 1);

        if dphi * oldphi < 0 % Sign change: eigenvalue bracketed
            k = k - dk;      % Step back
            dk = dk / 2;     % Halve the step size (bisection)
        end
      %  oldphi = dphi;     % must remove!!
    end
    kk(n) = k;            % Store the eigenvalue
    hold on;            % Keep previous plots
    plot(x, phi(:, 1));   % Plot the eigenfunction
end

tt = kk; % Return the eigenvalues.

    function yy = evfun(x, phi)
        global k
        yy = [phi(2);          % y' = dy/dx
              -k^2 * phi(1)];    % y'' = -k²y
